# Mini Projeto 8: Amostragem por importância

In [1]:
import matplotlib.pyplot as plt
import random as rnd
import numpy as np
import scipy.stats as stats

## Resultado da integral exata

In [2]:
func = lambda x: (np.exp(x) - 1)/(np.exp(1) - 1)

In [3]:
EXACT_INT = 0.41802

## Monte Carlo Tosco

In [29]:
def get_crude_montecarlo(N, func=func):
    randoms = [rnd.random() for i in range(N)]
    results = func(randoms)
    exp_res = sum(results)/N
    var_res = sum((results - exp_res)**2)/N
    ret = [exp_res, np.sqrt(var_res)]
    return ret

In [30]:
std_err = lambda x: (1/(len(x)-1)) * sum((x-np.mean(x))**2)

## Monte Carlo por Importância

In [31]:
cont_dist2 = lambda n,z: n*(z**(n-1))

In [32]:
def get_extreme_values_dist(N, N_samples):
    N_extreme = np.zeros(N_samples)

    for x in range(N_samples):
        X = []
        r = 1
        for i in range(N):
            X.append(rnd.random())
            r *= X[i]
            
        Z = max(X)
        if (r <= Z):
            N_extreme[x] += Z
        
    return N_extreme

In [33]:
def indicator_func(x, threshold):
    ret = []
    for i in x:
        if i < threshold:
            ret.append(1)
            
        else:
            ret.append(0)
    
    return ret

## Resultados

### Tosco: utilizando N=20 pontos

In [34]:
mc_0 = get_crude_montecarlo(20)

print("Monte Carlo Tosco: \n", "{}({})".format(mc_0[0], mc_0[1]))

Monte Carlo Tosco: 
 0.49159415367605314(0.34608731333318676)


Onde o primeiro termo representa o valor estimado $\hat\theta$ e entre parênteses, metade do intervalo de confiança $\sigma$

### Importância: utilizando N=20 pontos

In [23]:
n = 20

In [24]:
N1_ext = get_extreme_values_dist(1, n)
N2_ext = get_extreme_values_dist(2, n)
N3_ext = get_extreme_values_dist(3, n)
N4_ext = get_extreme_values_dist(4, n)

In [26]:
estim = (1/n)*sum(np.divide(np.multiply(func(N1_ext),indicator_func(N1_ext, 1)), cont_dist2(1,N1_ext)))
estim_var = sum((func(N1_ext) - estim)**2)/n
ret1 = [estim, 2*np.sqrt(estim_var)] 

estim = (1/n)*sum(np.divide(np.multiply(func(N2_ext),indicator_func(N2_ext, 1)), cont_dist2(2,N2_ext)))
estim_var = sum((func(N2_ext) - estim)**2)/n
ret2 = [estim, 2*np.sqrt(estim_var)] 

estim = (1/n)*sum(np.divide(np.multiply(func(N3_ext),indicator_func(N3_ext, 1)), cont_dist2(3,N3_ext)))
estim_var = sum((func(N3_ext) - estim)**2)/n
ret3 = [estim, 2*np.sqrt(estim_var)] 

estim = (1/n)*sum(np.divide(np.multiply(func(N4_ext),indicator_func(N4_ext, 1)), cont_dist2(4,N4_ext)))
estim_var = sum((func(N4_ext) - estim)**2)/n
ret4 = [estim, 2*np.sqrt(estim_var)] 

In [29]:
print("Monte Carlo Importância (k = 1): \n", "{}({})\n\n".format(ret1[0], ret1[1]))
print("Monte Carlo Importância (k = 2): \n", "{}({})\n\n".format(ret2[0], ret2[1]))
print("Monte Carlo Importância (k = 3): \n", "{}({})\n\n".format(ret3[0], ret3[1]))
print("Monte Carlo Importância (k = 4): \n", "{}({})\n\n".format(ret4[0], ret4[1]))

Monte Carlo Importância (k = 1): 
 0.48067773319853535(0.6359829756257793)


Monte Carlo Importância (k = 2): 
 0.41761751720702467(0.6716759753257928)


Monte Carlo Importância (k = 3): 
 0.4430637714804183(0.6927349046074152)


Monte Carlo Importância (k = 4): 
 0.3731222486370236(0.8369902947643326)




Onde o primeiro termo representa o valor estimado $\hat\theta$ e entre parênteses, metade do intervalo de confiança $\sigma$

### Resultados analíticos para as variâncias

Sendo a variância $V$ dada por

$$
V = \int_{0}^{1} \Bigg[\frac{h(x)f(x)}{g(x)}\Bigg]^2 g(x) dx - \theta^2
$$

E definindo

$$
h^*(X) = \frac{h(X)f(X)}{g(X)}
$$

Temos que

$$
V = \int_{0}^{1} h^*(x)^2 g(x) dx - \theta^2
$$

Onde $f(x)$ e $g(x)$ representam ambas densidades de probabilidades

Utilizando a função $g(x)$ dada por $g_k(x) = kx^{k-1}$, tal como utilizado para determinar os valores da integral

$$
\int_0^1 h(x)f(x) dx = \int_{0}^1 \frac{e^x - 1}{e - 1} dx
$$

Desta forma, temos que 

$$
V = \int_0^1 \Bigg[\frac{e^x - 1}{e - 1}\Bigg]^2 \frac{1}{kx^{k-1}} dx - \theta^2 \implies
$$

$$
V = \frac{1}{k(e - 1)^2} \int_0^1 \frac{(e^x - 1)^2}{x^{k-1}} dx - \theta^2
$$

Para k=1, o resultado converge, porém para k>1, precisamos utilizar aproximações numéricas. 

In [44]:
var_func1 = lambda x: ((np.exp(x) - 1)**2)
var_func2 = lambda x: np.divide(((np.exp(x) - 1)**2),(x))
var_func3 = lambda x: np.divide(((np.exp(x) - 1)**2),(np.multiply(x,x)))
var_func4 = lambda x: np.divide(((np.exp(x) - 1)**2),(np.multiply(np.multiply(x,x),x)))

In [65]:
mc_1 = get_crude_montecarlo(100, var_func1)
mc_2 = get_crude_montecarlo(100, var_func2)
mc_3 = get_crude_montecarlo(100, var_func3)
mc_4 = get_crude_montecarlo(100, var_func4)


print("Monte Carlo Tosco (k = 1): \n", "{}({})\n\n".format(mc_1[0], mc_1[1]))
print("Monte Carlo Tosco (k = 2): \n", "{}({})\n\n".format(mc_2[0], mc_2[1]))
print("Monte Carlo Tosco (k = 3): \n", "{}({})\n\n".format(mc_3[0], mc_3[1]))
print("Monte Carlo Tosco (k = 4): \n", "{}({})\n\n".format(mc_4[0], mc_4[1]))

Monte Carlo Tosco (k = 1): 
 0.6902901230314356(0.8415016059504008)


Monte Carlo Tosco (k = 2): 
 1.1256479249872684(0.8274228925758299)


Monte Carlo Tosco (k = 3): 
 1.7751458370245263(0.5592426402940807)


Monte Carlo Tosco (k = 4): 
 8.778661316940081(27.95050215742991)




Assim, temos que as varinâncias "analíticas" (entre aspas, já que apesar da fórmula decorrer de resultados analíticos, utilizamos ao fim monte carlo tosco para aproximar seu valor)

In [49]:
f = lambda k, int_res, theta: (1/(k*(np.exp(1) - 1)**2)) * int_res - theta**2

In [63]:
print("Para k = 1\nV = {}\n\n".format(f(1, mc_1[0], ret1[0])))
print("Para k = 2\nV = {}\n\n".format(f(2, mc_2[0], ret2[0])))
print("Para k = 3\nV = {}\n\n".format(f(3, mc_3[0], ret3[0])))
print("Para k = 4\nV = {}\n\n".format(f(4, mc_4[0], ret4[0])))

Para k = 1
V = 0.24748700185792014


Para k = 2
V = 0.1901277073572234


Para k = 3
V = 0.18014080851868375


Para k = 4
V = 0.5623599314440245




É importante notar que as igualdades acima devem ser interpretadas como aproximações, já que existem erros associados a cada um dos resultados que podem ser obtidos através da propagação de erro